In [4]:
!pip install catboost

In [5]:
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score
import lightgbm as lgb
from catboost import CatBoostClassifier, Pool
from sklearn.ensemble import GradientBoostingClassifier


In [6]:
df = pd.read_csv('bank-additional-full.csv', sep = ';')

In [7]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [8]:
df.shape

(14770, 21)

In [10]:
df.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     1
euribor3m         1
nr.employed       1
y                 1
dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14770 entries, 0 to 14769
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             14770 non-null  int64  
 1   job             14770 non-null  object 
 2   marital         14770 non-null  object 
 3   education       14770 non-null  object 
 4   default         14770 non-null  object 
 5   housing         14770 non-null  object 
 6   loan            14770 non-null  object 
 7   contact         14770 non-null  object 
 8   month           14770 non-null  object 
 9   day_of_week     14770 non-null  object 
 10  duration        14770 non-null  int64  
 11  campaign        14770 non-null  int64  
 12  pdays           14770 non-null  int64  
 13  previous        14770 non-null  int64  
 14  poutcome        14770 non-null  object 
 15  emp.var.rate    14770 non-null  float64
 16  cons.price.idx  14770 non-null  float64
 17  cons.conf.idx   14769 non-null 

In [29]:
X = df[['age', 'job', 'marital', 'education', 'housing', 'y']]

In [32]:
X = pd.get_dummies(X, X.columns[X.dtypes == 'object'])

In [34]:
X.head()

,age,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,...,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,housing_no,housing_unknown,housing_yes,y_no,y_yes
0,56,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,57,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,1,0,0,1,0
2,37,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,1,1,0
3,40,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
4,56,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,1,0,0,1,0


In [38]:
y = X.y_no
X.drop(columns = ['y_no', 'y_yes'], inplace = True)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

In [45]:
X_train.head()

,age,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,...,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,housing_no,housing_unknown,housing_yes
3345,29,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1
2472,46,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
13451,27,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
4022,35,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0
4070,35,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1


#Decision Tree

In [47]:
dTree_clf = DecisionTreeClassifier()

In [48]:
dTree_clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [49]:
y_pred2 = dTree_clf.predict(X_test)

In [54]:
print("Accuracy of Model::",accuracy_score(y_test,y_pred))

Accuracy of Model:: 0.9373730534867976


#XGBoost

In [55]:
xgb_classifier = xgb.XGBClassifier()

xgb_classifier.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [57]:
predictions_xgb = xgb_classifier.predict(X_test)

In [59]:
print("Accuracy of Model::",accuracy_score(y_test,predictions_xgb))

Accuracy of Model:: 0.9620853080568721


#LightGBM

In [61]:
train_dataset = lgb.Dataset(X_train, y_train)
test_dataset = lgb.Dataset(X_test, y_test)

In [66]:
lgb_classifier = lgb.train({"objective" : " binary", "verbosity": -1},
                           train_set = train_dataset, valid_sets=(test_dataset,), num_boost_round=10)

In [69]:
test_preds = lgb_classifier.predict(X_test)
train_preds = lgb_classifier.predict(X_train)

In [71]:
test_preds = [1 if pred > 0.5 else 0 for pred in test_preds]
train_preds = [1 if pred > 0.5 else 0 for pred in train_preds]

In [77]:
print("Test Accuracy: ", accuracy_score(y_test, test_preds))
print("Train Accuracy: ", accuracy_score(y_train, train_preds))

Test Accuracy:  0.962762356127285
Train Accuracy:  0.9597156398104265


#CatBoost

In [81]:
model = CatBoostClassifier(iterations = 2,
                           depth=2,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)

model.fit(X_train, y_train)

0:	learn: 0.1687741	total: 50.4ms	remaining: 50.4ms
1:	learn: 0.1685195	total: 53.5ms	remaining: 0us


In [83]:
test_preds_cb = model.predict(X_test)
preds_proba = model.predict_proba(X_test)
print("class = ", test_preds_cb)
print("proba = ", preds_proba)

class =  [1 1 1 ... 1 1 1]
proba =  [[0.03958858 0.96041142]
 [0.04654198 0.95345802]
 [0.03628006 0.96371994]
 ...
 [0.03958858 0.96041142]
 [0.03628006 0.96371994]
 [0.03650134 0.96349866]]


In [84]:
print(accuracy_score(test_preds_cb, y_test))

0.962762356127285


In [86]:
model_sk = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)
model_sk.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=0)

In [88]:
model_sk.score(X_test, y_test)

0.962762356127285